数据读进来

In [50]:
import pandas as pd
df = pd.read_csv('..\\..\\data\\猫鸭账单_2025年.csv', engine='python', encoding='utf-8-sig')

# 1 清洗数据格式
## 1.1 清洗表示“钱”的数据
现在的流水是一个`object`：流水    432 non-null object

但是我们如果想根据流水做一些操作，它必须得是一个数值类型

所以就想办法把他变成一个数值类型

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 7 columns):
摘要    432 non-null object
日期    432 non-null object
分类    432 non-null object
金额    432 non-null float64
收支    432 non-null object
流水    432 non-null object
月份    432 non-null object
dtypes: float64(1), object(6)
memory usage: 23.7+ KB


## Series的apply方法——对Series里的每个元素应用某种操作，并返回新的Series
用`apply`函数对指定的数据列应用某种操作

现在的“流水”的样子是`"¥21,953.55	"`我们希望它变成的样子是`float`类型的`21953.55`

那么我们做的事情就是先把这个字符串里的无效字符去掉：
1. 去掉`￥`人民币符号
2. 去掉所有的`,`分隔符

接下来，把清洗好的“字符串”变成`float`类型，这一步我们只要使用Python的`float()`函数转换一下就可以了

In [8]:
df[df['金额']>1000].head()

,摘要,日期,分类,金额,收支,流水,月份
86,小橘救治费用,2025/01/15,宠物,30000.00,支出,"-¥30,000.00",1 月
160,小猫1月工资,2025/01/24,工资,21953.55,收入,"¥21,953.55",1 月
161,小猫1月股权激励计划,2025/01/24,工资,6778.75,收入,"¥6,778.75",1 月
162,小猫给家里转账,2025/01/29,家里,4000.00,支出,"-¥4,000.00",1 月
169,小橘救助1月众筹收入（不含追梦筹）,2025/01/31,宠物,3307.88,收入,"¥3,307.88",1 月


In [18]:
a = " ¥21,953.55 "
b = float(a.replace("¥", "").replace(",", "").strip())
b

21953.55

In [19]:
type(b)

float

接下来我们对所有的“流水”都应用这一操作，用到`apply`方法

`pandas.core.series.Series`的`apply`方法接收一个自定义的函数，然后它会对这个`Series`里的每个元素都应用这个函数，将它转换成函数返回值给出的样子

但是它不是一个原地操作，所以要赋值回去给之前的DataFrame里的对应列（`Series`）

> 注意：在我写回CSV文件之前，即使修改了读出的DataFrame，也不会改对应的CSV文件

In [56]:
def format_flow(a):
    return float(a.replace("¥", "").replace(",", "").strip())
    
df["流水"] = df["流水"].apply(format_flow)

In [30]:
df.head()

,摘要,日期,分类,金额,收支,流水,月份
0,晚餐潮牛牛,2025/01/01,饮食,182.00,支出,-182.00,1 月
1,午餐鲜烫牛肉粉,2025/01/01,饮食,34.90,支出,-34.90,1 月
2,百果园充值,2025/01/01,水果,94.00,支出,-94.00,1 月
3,美团买菜,2025/01/02,饮食,42.88,支出,-42.88,1 月
4,一次性棉球,2025/01/01,宠物,0.60,支出,-0.60,1 月


## 1.2 清洗表示“时间”的数据

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 7 columns):
摘要    432 non-null object
日期    432 non-null object
分类    432 non-null object
金额    432 non-null float64
收支    432 non-null object
流水    432 non-null float64
月份    432 non-null object
dtypes: float64(2), object(5)
memory usage: 23.7+ KB


现在，我们的“日期”这一列其实还是普通的字符串，会有什么不好的呢？就是计算一些时间相关的东西比较麻烦，比如我想知道两个日期之间相差多少天，或者相差多少个月，拿字符串去计算是非常麻烦的。

但是Python其实提供了表示时间的数据类型，我们只要想办法把它转换成对应的类型就可以了！

In [32]:
"2025/01/04" - "2025/01/02"

TypeError: unsupported operand type(s) for -: 'str' and 'str'

## Python中的datetime模块介绍
Python里经常用这个模块表示时间

In [34]:
from datetime import datetime

In [40]:
dt_format = "%Y/%m/%d" # 格式串，告诉datetime我要以怎样的方式提供给你时间字符串
dt1 = datetime.strptime("2025/01/02", dt_format)
dt2 = datetime.strptime("2025/01/04", dt_format)

In [43]:
dt_diff = dt2 - dt1
dt_diff.days

2

### 1.2.1 第一种方法
手写模式串，然后使用`datetime.strptime`进行按模式读取的转换

In [45]:
def format_time(t):
    return datetime.strptime(t, "%Y/%m/%d")
df['日期'] = df['日期'].apply(format_time)

In [47]:
df['日期'].head()

0   2025-01-01
1   2025-01-01
2   2025-01-01
3   2025-01-02
4   2025-01-01
Name: 日期, dtype: datetime64[ns]

### 1.2.2 第二种方法
针对比较规整的日期字符串，可以直接使用`pandas`模块提供的`to_datetime`方法，该方法接收一个`Series`，然后将这个`Series`里的所有日期字符串数据都变成`datetime`类型，最终返回一个新的`Series`

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 7 columns):
摘要    432 non-null object
日期    432 non-null object
分类    432 non-null object
金额    432 non-null float64
收支    432 non-null object
流水    432 non-null object
月份    432 non-null object
dtypes: float64(1), object(6)
memory usage: 23.7+ KB


In [54]:
df['日期'] = pd.to_datetime(df['日期'])

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 7 columns):
摘要    432 non-null object
日期    432 non-null datetime64[ns]
分类    432 non-null object
金额    432 non-null float64
收支    432 non-null object
流水    432 non-null float64
月份    432 non-null object
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 23.7+ KB


我们把月份变成正确的月份数据，直接用`datetime`组成的`Series`访问它的`dt.month`属性就是这个`datetime`的月份组成的`Series`

In [61]:
df['月份'] = df['日期'].dt.month

In [62]:
df.head()

,摘要,日期,分类,金额,收支,流水,月份
0,晚餐潮牛牛,2025-01-01,饮食,182.00,支出,-182.00,1
1,午餐鲜烫牛肉粉,2025-01-01,饮食,34.90,支出,-34.90,1
2,百果园充值,2025-01-01,水果,94.00,支出,-94.00,1
3,美团买菜,2025-01-02,饮食,42.88,支出,-42.88,1
4,一次性棉球,2025-01-01,宠物,0.60,支出,-0.60,1
